In [1]:
# Import packages
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd
# Download data and extract the table with Beautiful Soup
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url,'lxml')
My_table = soup.find('table',{'class':'wikitable sortable'})
# Extract columns and rows from the Wikipedia table
A=[]
B=[]
C=[]
 
for row in My_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

In [2]:
# Create Pandas dataframe
df=pd.DataFrame(A,columns=['Postal Code'])
df['Borough']=B
df['Neighborhood']=C

In [3]:
# Drop boroughs with the value 'Not assigned'
df = df[df.Borough != 'Not assigned']

In [4]:
# Import Numpy
import numpy as np
# Merge rows based on same Postal Code area
df.groupby('Postal Code').sum().replace(0,np.nan)
# Reset index
df = df.reset_index(drop=True)

In [5]:
# Change of 'Not assigned' neighborhood values to borough's values
df = df.groupby(['Postal Code', 'Borough'])['Neighborhood'].apply(','.join).reset_index()

In [6]:
# Print number of rows and columns
df.shape

(103, 3)

In [7]:
df.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood\n,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
# Upload latitude and longitude using .csv file
df2 = pd.read_csv(r'C:\Users\Dade\Downloads\Geospatial_Coordinates.csv')
# Merge databases
df = pd.merge(df, df2)
df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood\n,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park\n,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West\n",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West\n",43.692657,-79.264848


In [9]:
# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim 
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [10]:
# Import Folium and create toronto map

import folium
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to the map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto